# Citi Bike: Usage patterns + insurer decision assets

This notebook is built to work with the repo Make targets (e.g.make report-both YEARS="YYYY YYYY..." MONTHS="1 2 .. 12")
It **always prefers the current run** (your `summaries/<RUN_TAG>/` folder) when you pass `YEARS/MONTHS`, so the report reflects exactly what you ingested.


## How to run (recommended)

From the repo root:
```bash
make all-both YEARS="YYYY YYYY..." MONTHS="1 2 .. 12"

The Makefile sets environment variables (e.g. `CITIBIKE_PARQUET_DIR`, `CITIBIKE_YEARS`, `CITIBIKE_MONTHS`) which this notebook reads.


In [ ]:
# --- Setup (STRICT + explicit risk variants): repo paths, run dirs, safe CSV loads ---

from __future__ import annotations

from pathlib import Path
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

# Optional display in notebooks
try:
    from IPython.display import display, Markdown
except Exception:
    display = print
    Markdown = lambda x: x

# nbconvert-friendly
try:
    get_ipython().run_line_magic("matplotlib", "inline")
except Exception:
    pass
plt.ioff()

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start, *start.parents]:
        if (p / "Makefile").exists():
            return p
    raise FileNotFoundError(
        "Could not find repo root (Makefile) by walking up from:\n"
        f"  CWD={Path.cwd().resolve()}"
    )

def _parse_int_list(val: str | None):
    if val is None:
        return None
    s = str(val).strip()
    if not s:
        return None
    parts = re.split(r"[,\s]+", s)
    out = []
    for p in parts:
        if not p:
            continue
        try:
            out.append(int(p))
        except Exception:
            pass
    return out or None

def read_csv_strict(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Missing required CSV: {path}")
    return pd.read_csv(path)

def read_csv_optional(path: Path) -> pd.DataFrame | None:
    return pd.read_csv(path) if path.exists() else None

REPO_ROOT = find_repo_root(Path.cwd())
SUMMARIES_ROOT = REPO_ROOT / "summaries"

PARQUET_DIR_ENV = (os.environ.get("CITIBIKE_PARQUET_DIR") or "").strip()
RUN_DIR_ENV     = (os.environ.get("CITIBIKE_RUN_DIR") or "").strip()
MODE_ENV        = (os.environ.get("CITIBIKE_MODE") or os.environ.get("MODE") or "").strip().lower()

YEARS_FILTER  = _parse_int_list(os.environ.get("CITIBIKE_YEARS")  or os.environ.get("YEARS"))
MONTHS_FILTER = _parse_int_list(os.environ.get("CITIBIKE_MONTHS") or os.environ.get("MONTHS"))

PARQUET_DIR = Path(PARQUET_DIR_ENV) if PARQUET_DIR_ENV else Path()

if RUN_DIR_ENV:
    RUN_DIR = Path(RUN_DIR_ENV)
else:
    run_tag = PARQUET_DIR.name if str(PARQUET_DIR).strip() else ""
    RUN_DIR = (SUMMARIES_ROOT / run_tag) if run_tag else Path()

# Resolve relative paths against repo root
if str(RUN_DIR).strip() and (not RUN_DIR.is_absolute()):
    RUN_DIR = (REPO_ROOT / RUN_DIR).resolve()
if str(PARQUET_DIR).strip() and (not PARQUET_DIR.is_absolute()):
    PARQUET_DIR = (REPO_ROOT / PARQUET_DIR).resolve()

# Strict folder checks
if not SUMMARIES_ROOT.exists():
    raise FileNotFoundError(f"Expected summaries/ at: {SUMMARIES_ROOT} (run make summarize first)")

if not RUN_DIR.exists():
    raise FileNotFoundError(f"RUN_DIR not found: {RUN_DIR} (run make summarize first)")

if str(PARQUET_DIR).strip() and (not PARQUET_DIR.exists()):
    raise FileNotFoundError(f"PARQUET_DIR not found: {PARQUET_DIR} (run make ingest first)")

# Required run CSVs
REQUIRED_RUN_FILES = [
    "citibike_trips_by_year.csv",
    "citibike_trips_by_month.csv",
    "citibike_trips_by_dow.csv",
    "citibike_trips_by_hour.csv",
]
missing = [f for f in REQUIRED_RUN_FILES if not (RUN_DIR / f).exists()]
if missing:
    raise FileNotFoundError(
        f"Missing required summary CSVs in:\n  {RUN_DIR}\nMissing: {missing}\n"
        "Run: make summarize-only (or make summarize / make all)."
    )

print("REPO_ROOT:", REPO_ROOT)
print("RUN_DIR:", RUN_DIR)
print("PARQUET_DIR:", PARQUET_DIR if str(PARQUET_DIR).strip() else "(not set)")
print("MODE (env):", MODE_ENV or "(not set)")
print("YEARS_FILTER:", YEARS_FILTER, "MONTHS_FILTER:", MONTHS_FILTER)

# Figures folder
FIG_DIR = REPO_ROOT / "reports" / RUN_DIR.name / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)
print("FIG_DIR:", FIG_DIR)

def savefig(filename: str):
    out = FIG_DIR / filename
    plt.savefig(out, dpi=200, bbox_inches="tight")
    print("Saved:", out)

# Load per-run summaries
df_year  = read_csv_strict(RUN_DIR / "citibike_trips_by_year.csv")
df_month = read_csv_strict(RUN_DIR / "citibike_trips_by_month.csv")
df_dow   = read_csv_strict(RUN_DIR / "citibike_trips_by_dow.csv")
df_hour  = read_csv_strict(RUN_DIR / "citibike_trips_by_hour.csv")

# Optional per-run outputs
df_station = read_csv_optional(RUN_DIR / "citibike_station_exposure.csv")

# Risk variants (overall + new per-year/per-month if present)
df_risk_overall = read_csv_optional(RUN_DIR / "station_risk_exposure_plus_crashproximity.csv")
df_risk_year    = read_csv_optional(RUN_DIR / "station_risk_exposure_plus_crashproximity_by_year.csv")
df_risk_ym      = read_csv_optional(RUN_DIR / "station_risk_exposure_plus_crashproximity_by_year_month.csv")

# Backward compat: keep df_risk pointing to "best available"
df_risk = df_risk_year if df_risk_year is not None else df_risk_overall

# Mode detection
mode = (
    str(df_year["mode"].iloc[0]).lower()
    if ("mode" in df_year.columns and len(df_year))
    else (MODE_ENV or "unknown")
)
print("Detected mode:", mode)

# Apply optional filters defensively
def _filter_year_month(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if YEARS_FILTER is not None and "year" in out.columns:
        out["year"] = pd.to_numeric(out["year"], errors="coerce")
        out = out[out["year"].isin(YEARS_FILTER)]
    if MONTHS_FILTER is not None and "month" in out.columns:
        out["month"] = pd.to_numeric(out["month"], errors="coerce")
        out = out[out["month"].isin(MONTHS_FILTER)]
    return out

df_year  = _filter_year_month(df_year)
df_month = _filter_year_month(df_month)
df_dow   = _filter_year_month(df_dow)
df_hour  = _filter_year_month(df_hour)

# Helpful run label
run_label = RUN_DIR.name

print("\nRisk inputs found:")
print(" - df_risk_overall:", "YES" if df_risk_overall is not None else "NO")
print(" - df_risk_year   :", "YES" if df_risk_year is not None else "NO")
print(" - df_risk_ym     :", "YES" if df_risk_ym is not None else "NO")
print("Using df_risk = ", "df_risk_year" if df_risk is df_risk_year else ("df_risk_overall" if df_risk is df_risk_overall else "None"))


## Executive summary (optional)

If you generated `summary_highlights.md` for this run, we show it here.


In [ ]:
# --- Quick text highlights from summarize script (if present) ---
from pathlib import Path

# Prefer the variable if it exists, else derive from RUN_DIR (which setup should define)
hp = None
if "highlights_path" in globals():
    try:
        hp = highlights_path
    except Exception:
        hp = None

if hp is None:
    if "RUN_DIR" in globals():
        hp = Path(RUN_DIR) / "summary_highlights.md"
    else:
        # Last resort: try repo-relative "summaries/latest"
        hp = Path("summaries/latest/summary_highlights.md").resolve()

if hp.exists():
    txt = hp.read_text(encoding="utf-8", errors="ignore")
    try:
        display(Markdown(txt))
    except Exception:
        print(txt)
else:
    print("No summary_highlights.md found at:", hp)



## 1) Yearly usage (comparison across years)

In [ ]:
# --- Trips by year (THIS RUN) ---
g = df_year.copy()

if "year" in g.columns:
    # Clean and prepare data
    g["year"] = pd.to_numeric(g["year"], errors="coerce")
    g = g.dropna(subset=["year"]).sort_values("year").reset_index(drop=True)
    
display(g)

# Plot
plt.figure(figsize=(10, 6))

if "year" in g.columns and "trips" in g.columns:
    # Extract values as plain lists (no pandas magic)
    years = [int(y) for y in g["year"]]
    trips = [int(t) for t in g["trips"]]
    
    # Plot with explicit year labels
    plt.plot(years, trips, marker="o", linewidth=2, markersize=8, color='#2E86AB')
    plt.xticks(years, [str(y) for y in years])  # Explicit year labels
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Add value labels on points
    for year, trip in zip(years, trips):
        plt.text(year, trip, f'{trip/1e6:.1f}M', 
                ha='center', va='bottom', fontsize=9)

plt.title(f"Trips by year — mode={mode} ({run_label})", fontsize=14, fontweight='bold')
plt.xlabel("Year", fontsize=12)
plt.ylabel("Trips", fontsize=12)
plt.tight_layout()

savefig("01_trips_by_year.png")
plt.show()


## 2) Month patterns (comparison)

In [ ]:
# --- Trips by month (THIS RUN) ---
m = df_month.copy()

# Normalize month labels
if "month" in m.columns:
    m["month"] = pd.to_numeric(m["month"], errors="coerce").astype("Int64")
if "year" in m.columns:
    m["year"] = pd.to_numeric(m["year"], errors="coerce").astype("Int64")

display(m.sort_values([c for c in ["year", "month"] if c in m.columns]))

# If multiple years, show stacked-ish bars by year-month
if {"year", "month", "trips"}.issubset(m.columns):
    m2 = m.dropna(subset=["year","month"]).copy()
    m2["ym"] = m2["year"].astype(int).astype(str) + "-" + m2["month"].astype(int).astype(str).str.zfill(2)

    plt.figure(figsize=(10,4))
    plt.bar(m2["ym"], m2["trips"])
    plt.title(f"Trips by year-month — mode={mode} ({run_label})")
    plt.xlabel("Year-Month")
    plt.ylabel("Trips")
    plt.xticks(rotation=75, ha="right")
    plt.tight_layout()
    savefig("02_trips_by_year_month.png")
    plt.show()
else:
    print("Expected columns not present for trips-by-month plot. Have:", list(m.columns))


In [ ]:
# --- Month-of-year seasonality (line chart per year) ---
m = df_month.copy()
if not {"year","month","trips"}.issubset(m.columns):
    print("Month table missing required columns:", list(m.columns))
else:
    m["year"]  = pd.to_numeric(m["year"], errors="coerce").astype("Int64")
    m["month"] = pd.to_numeric(m["month"], errors="coerce").astype("Int64")
    m = m.dropna(subset=["year","month"]).copy()
    m = m.sort_values(["year","month"])

    pivot = m.pivot_table(index="month", columns="year", values="trips", aggfunc="sum").sort_index()
    display(pivot)

    plt.figure(figsize=(8,4))
    for y in pivot.columns:
        plt.plot(pivot.index.astype(int), pivot[y], marker="o", label=str(int(y)))
    plt.title(f"Trips by month-of-year — mode={mode} ({run_label})")
    plt.xlabel("Month")
    plt.ylabel("Trips")
    plt.xticks(range(1,13))
    plt.legend(title="Year", ncol=2, fontsize=8)
    plt.tight_layout()
    savefig("03_month_by_year_lines.png")
    plt.show()


## 3) Day-of-week patterns (comparison)

In [ ]:
# --- Day-of-week patterns (comparison across years; robust even if dow_name missing) ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

if df_dow is None or len(df_dow) == 0:
    print("df_dow is None/empty; skipping DOW comparison.")
else:
    d = df_dow.copy()

    # ---- Required columns ----
    req = {"dow", "trips"}
    missing = [c for c in req if c not in d.columns]
    if missing:
        raise KeyError(f"df_dow missing required columns {missing}. Have: {list(d.columns)}")

    # Ensure year exists for "comparison"
    if "year" not in d.columns:
        print("df_dow has no 'year' column; cannot do year comparison.")
        display(d.head(20))
    else:
        # ---- Clean types ----
        d["dow"] = pd.to_numeric(d["dow"], errors="coerce")
        d["trips"] = pd.to_numeric(d["trips"], errors="coerce").fillna(0)
        d["year"] = pd.to_numeric(d["year"], errors="coerce")
        d = d.dropna(subset=["dow", "year"]).copy()
        d["dow"] = d["dow"].astype(int)
        d["year"] = d["year"].astype(int)

        # ---- Add dow_name if missing (deterministic mapping) ----
        if "dow_name" not in d.columns:
            dow_map = {
                0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday",
                4: "Friday", 5: "Saturday", 6: "Sunday",
            }
            d["dow_name"] = d["dow"].map(dow_map).fillna(d["dow"].astype(str))

        # ---- Add week_part if missing ----
        if "week_part" not in d.columns:
            d["week_part"] = np.where(d["dow"] >= 5, "weekend", "weekday")

        years = sorted(d["year"].unique().tolist())
        print("Years present in df_dow:", years)

        # ---- TABLE 1: pivot (dow_name x year) ----
        pivot = (
            d.pivot_table(index=["dow", "dow_name"], columns="year", values="trips", aggfunc="sum")
             .sort_index(level=0)
        )
        print("\nTrips by day-of-week (pivot):")
        display(pivot)

        # ---- TABLE 2: shares within each year (percent of yearly total) ----
        shares = (
            d.groupby(["year", "dow", "dow_name"], as_index=False)["trips"]
             .sum()
             .sort_values(["year", "dow"])
        )
        shares["pct_of_year"] = shares["trips"] / shares.groupby("year")["trips"].transform("sum") * 100.0
        share_pivot = (
            shares.pivot_table(index=["dow", "dow_name"], columns="year", values="pct_of_year", aggfunc="first")
                  .sort_index(level=0)
        )
        print("\nShare of trips within each year (%):")
        display(share_pivot.round(2))

        # ---- PLOT 1: absolute trips per DOW for each year ----
        plt.figure(figsize=(10, 4))
        x_labels = [f"{n}" for n in pivot.index.get_level_values("dow_name")]
        x = np.arange(len(x_labels))

        for yy in years:
            if yy in pivot.columns:
                plt.plot(x, pivot[yy].values, marker="o", label=str(yy))

        plt.xticks(x, x_labels, rotation=30, ha="right")
        plt.xlabel("Day of Week")
        plt.ylabel("Trips")
        plt.title(f"Trips by Day of Week — year comparison — mode={mode} ({run_label})")
        plt.grid(alpha=0.25, axis="y")
        plt.legend(title="Year")
        plt.tight_layout()
        savefig("10_dow_year_comparison_abs.png")
        plt.show()

        # ---- PLOT 2: % share per DOW for each year (better for pattern comparison) ----
        plt.figure(figsize=(10, 4))
        for yy in years:
            if yy in share_pivot.columns:
                plt.plot(x, share_pivot[yy].values, marker="o", label=str(yy))

        plt.xticks(x, x_labels, rotation=30, ha="right")
        plt.xlabel("Day of Week")
        plt.ylabel("Share of trips within year (%)")
        plt.title(f"Day-of-week pattern (% within year) — year comparison — mode={mode} ({run_label})")
        plt.grid(alpha=0.25, axis="y")
        plt.legend(title="Year")
        plt.tight_layout()
        savefig("10_dow_year_comparison_pct.png")
        plt.show()

        # ---- OPTIONAL: weekday vs weekend bar per year ----
        wp = (
            d.groupby(["year", "week_part"], as_index=False)["trips"]
             .sum()
             .sort_values(["year", "week_part"])
        )
        wp["pct"] = wp["trips"] / wp.groupby("year")["trips"].transform("sum") * 100.0

        plt.figure(figsize=(8, 4))
        # make consistent order
        order = ["weekday", "weekend"]
        for yy in years:
            sub = wp[wp["year"] == yy].copy()
            sub["week_part"] = pd.Categorical(sub["week_part"], categories=order, ordered=True)
            sub = sub.sort_values("week_part")
            plt.plot(sub["week_part"].astype(str), sub["pct"].values, marker="o", label=str(yy))

        plt.xlabel("Week Part")
        plt.ylabel("Share of trips within year (%)")
        plt.title(f"Weekday vs Weekend share — year comparison — mode={mode} ({run_label})")
        plt.grid(alpha=0.25, axis="y")
        plt.legend(title="Year")
        plt.tight_layout()
        savefig("10_weekday_weekend_year_comparison_pct.png")
        plt.show()


## 4) Hour-of-day patterns (comparison)

In [ ]:
# --- Trips by hour: compare years (table + plot) ---

import pandas as pd
import matplotlib.pyplot as plt

h = df_hour.copy()

hour_col = "hour" if "hour" in h.columns else None
trips_col = "trips" if "trips" in h.columns else None
year_col  = "year" if "year" in h.columns else None

part_col = None
for c in ["week_part", "segment", "is_weekend"]:
    if c in h.columns:
        part_col = c
        break

if not (hour_col and trips_col and year_col):
    print("Hour table missing required columns. Have:", list(h.columns))
else:
    # Clean
    h[hour_col] = pd.to_numeric(h[hour_col], errors="coerce")
    h[trips_col] = pd.to_numeric(h[trips_col], errors="coerce")
    h[year_col] = pd.to_numeric(h[year_col], errors="coerce")
    h = h.dropna(subset=[hour_col, trips_col, year_col]).copy()
    h[hour_col] = h[hour_col].astype(int)
    h[year_col] = h[year_col].astype(int)

    years = sorted(h[year_col].unique().tolist())
    print("Years present in df_hour:", years)

    # ---- TABLES ----
    print("\nSample rows per year (first 8 hours for each year):")
    display(
        h.sort_values([year_col, hour_col])
         .groupby(year_col, as_index=False)
         .head(8)
    )

# #    if part_col:
# #        print(f"\nPivot table: trips by hour × year × {part_col}")
#         pivot = (
#             h.pivot_table(index=[hour_col, part_col], columns=year_col, values=trips_col, aggfunc="sum")
#              .sort_index()
#         )
#         display(pivot)
#     else:
#         print("\nPivot table: trips by hour × year")
#         pivot = (
#             h.pivot_table(index=hour_col, columns=year_col, values=trips_col, aggfunc="sum")
#              .sort_index()
#         )
#         display(pivot)

    # ---- PLOT ----
    plt.figure(figsize=(10,4))
    if part_col:
        for (yy, seg), sub in h.groupby([year_col, part_col]):
            sub = sub.sort_values(hour_col)
            plt.plot(sub[hour_col], sub[trips_col], marker="o", label=f"{yy} | {seg}")
    else:
        for yy, sub in h.groupby(year_col):
            sub = sub.sort_values(hour_col)
            plt.plot(sub[hour_col], sub[trips_col], marker="o", label=str(yy))

    plt.title(f"Trips by hour (year comparison) — mode={mode} ({run_label})")
    plt.xlabel("Hour")
    plt.ylabel("Trips")
    plt.xticks(range(0,24,1))
    plt.legend()
    plt.tight_layout()
    savefig("05_trips_by_hour_year_comparison.png")
    plt.show()


## 6) Crash proximity / risk proxy (optional)

### 6.1 Station-level prioritization (make the proxy usable)

The raw file includes **crash counts within 250m/500m** of each station plus **trip volume**.  
To avoid misleading outliers (e.g., stations with only a handful of trips), we:

- keep IDs as **strings** (so `2231.10` doesn’t turn into `2231.1`)
- recompute crash-rates per 100k trips (defensive)
- rank **exposure** (most trips) separately from **risk rate** (filtered to stations with enough trips)

You can tune `MIN_TRIPS_FOR_RATE` depending on how conservative you want to be.


### 6.2 Alternative Risk Metrics (Intuitive Formats)

The standard `per_100k_trips` metric is useful for technical analysis, but can be hard to interpret.
This section adds alternative formats that are more intuitive for different audiences:

- **Percentage** (crash_rate_pct): Good for executives
- **Trips per crash**: Most intuitive for general public
- **Risk level**: Classification (Very Low → Very High)
- **Risk score**: 0-100 ranking score

In [ ]:
# ---  Crash proximity risk proxy (NYC only): YEARLY comparison ---

import os
import re
import pandas as pd
import matplotlib.pyplot as plt

MIN_TRIPS_FOR_CREDIBLE_DISPLAY = 5000
TOP_N = 20


from pathlib import Path
        
if 'RUN_DIR' in globals():
            # Find any scorecard file
            scorecard_files = list(Path(RUN_DIR).glob("axa_partner_scorecard_*m.csv"))
            if scorecard_files:
                df_score = pd.read_csv(scorecard_files[0])
            else:
                print("   No scorecard files found in RUN_DIR")
                df_score = None     

# Only proceed if we have data
if df_score is not None and not df_score.empty:
    print(f"Using scorecard with {len(df_score):,} rows")
    
    # Normalize column names if needed
    if "station_id" not in df_score.columns and "start_station_id" in df_score.columns:
        df_score = df_score.rename(columns={"start_station_id": "station_id"})
    if "station_name" not in df_score.columns and "start_station_name" in df_score.columns:
        df_score = df_score.rename(columns={"start_station_name": "station_name"})
    
    # Check if we have year-level data
    if "year" not in df_score.columns:
        print("Scorecard has no 'year' column (overall aggregate only).")
    else:
        # Use PRE-COMPUTED EB values from scorecard
        required_cols = ["eb_risk_rate_per_100k_trips", "expected_incidents_proxy"]
        missing = [c for c in required_cols if c not in df_score.columns]
        
        if missing:
            print(f"WARNING: Scorecard missing EB columns: {missing}")
            print("Did build_axa_scorecard.py run successfully?")
        else:
            r = df_score.copy()
            
            # Filter to credible stations for display
            r["trips"] = pd.to_numeric(r.get("exposure_trips", r.get("trips", 0)), errors="coerce")
            r = r[r["trips"] >= MIN_TRIPS_FOR_CREDIBLE_DISPLAY].copy()
            
            years = sorted(r["year"].dropna().unique().tolist())
            
            print(f"\n=== Yearly Risk Analysis (using scorecard EB estimates) ===")
            print(f"Stations with ≥{MIN_TRIPS_FOR_CREDIBLE_DISPLAY:,} trips")
            
            yearly_burden = []
            
            for yy in years:
                ry = r[r["year"] == yy].copy()
                
                if ry.empty:
                    continue
                
                print(f"\n--- Year {int(yy)} ---")
                print(f"Top {TOP_N} by exposure:")
                display(ry.sort_values("trips", ascending=False).head(TOP_N)[
                    ["mode", "year", "station_id", "station_name", "trips"]
                ])
                
                print(f"Top {TOP_N} by EB risk rate:")
                display(ry.sort_values("eb_risk_rate_per_100k_trips", ascending=False).head(TOP_N)[
                    ["mode", "year", "station_id", "station_name", "trips", 
                     "crash_count", "eb_risk_rate_per_100k_trips"]
                ])
                
                # Yearly burden (sum of expected incidents)
                total_burden = ry["expected_incidents_proxy"].sum()
                yearly_burden.append({"year": int(yy), "eb_expected_crashes": total_burden})
            
            # Plot yearly burden
            if yearly_burden:
                yb = pd.DataFrame(yearly_burden).sort_values("year")
                
                plt.figure(figsize=(9, 4))
                plt.bar(yb["year"].astype(str), yb["eb_expected_crashes"].values)
                plt.title(f"Yearly EB crash-proxy burden — mode={mode}")
                plt.xlabel("Year")
                plt.ylabel("Expected incidents (sum across stations)")
                plt.tight_layout()
                savefig("09_yearly_EB_crash_proxy_burden.png")
                plt.show()
                
                print(f"\nTotal EB expected crashes by year:")
                display(yb)
else:
    print(" Skipping yearly risk analysis - no scorecard data available")

In [ ]:
# --- Exposure vs Risk "Zones" (EB-smoothed quadrant view, radius-aware) ---
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

MIN_TRIPS_FOR_ZONES = 5000
SCALE = 100000.0

_RADIUS_RE = re.compile(r"^\s*(\d+(?:\.\d+)?)\s*(m|km)?\s*$", re.IGNORECASE)

def parse_radius_to_m(raw: str) -> int:
    s = (raw or "").strip().lower()
    if s in ("", "auto", "max"):
        return -1
    m = _RADIUS_RE.match(s)
    if not m:
        return -1
    val = float(m.group(1))
    unit = (m.group(2) or "m").lower()
    meters = val * (1000.0 if unit == "km" else 1.0)
    return int(round(meters)) if meters > 0 else -1

def extract_radius_from_scorecard(df: pd.DataFrame) -> int | None:
    """Extract radius from crash_count column or scoring_strategy"""
    # Try to find crashes_within_{R}m column in raw risk data
    for c in df.columns:
        mm = re.match(r"^crashes_within_(\d+)m$", str(c))
        if mm:
            return int(mm.group(1))
    
    # Try to extract from scoring_strategy if present
    if "scoring_strategy" in df.columns and len(df) > 0:
        # scoring_strategy might contain info about radius used
        pass
    
    return None

# Determine which data source to use
use_scorecard = False
radius_used = None

if df_score is not None and len(df_score) > 0:
    print(" Using SCORECARD data (EB-smoothed)")
    r = df_score.copy()
    use_scorecard = True
    
    # Normalize column names
    if "station_id" not in r.columns and "start_station_id" in r.columns:
        r = r.rename(columns={"start_station_id": "station_id"})
    if "station_name" not in r.columns and "start_station_name" in r.columns:
        r = r.rename(columns={"start_station_name": "station_name"})
    
    # Get exposure (scorecard uses exposure_trips)
    if "exposure_trips" in r.columns:
        r["trips"] = pd.to_numeric(r["exposure_trips"], errors="coerce")
    elif "trips" not in r.columns:
        print("ERROR: No trips/exposure_trips column in scorecard")
        r = None
    
    # Use EB-smoothed risk rate if available
    if r is not None and "eb_risk_rate_per_100k_trips" in r.columns:
        r["risk_rate"] = pd.to_numeric(r["eb_risk_rate_per_100k_trips"], errors="coerce")
        risk_metric_name = "EB-smoothed risk rate per 100k trips"
    elif r is not None and "risk_rate_per_100k_trips" in r.columns:
        r["risk_rate"] = pd.to_numeric(r["risk_rate_per_100k_trips"], errors="coerce")
        risk_metric_name = "Raw risk rate per 100k trips"
        print("  Using raw risk rate (EB column not found)")
    else:
        print("ERROR: No risk rate column found in scorecard")
        r = None
    
    # Try to determine radius from scorecard
    if r is not None and "crash_count" in r.columns:
        # The scorecard was built from a specific radius
        # Try to extract it from the filename or strategy
        env_raw = os.environ.get("AXA_RADIUS", "auto")
        radius_used = parse_radius_to_m(env_raw)
        if radius_used == -1:
            # Parse from scorecard filename if loaded via glob
            radius_used = 500  # default assumption
        risk_metric_name = f"{risk_metric_name} ({radius_used}m radius)"

elif df_risk is not None:
    print("  FALLBACK: Using RAW risk data (no scorecard available)")
    r = df_risk.copy()
    
    # Normalize column names
    if "station_id" not in r.columns and "start_station_id" in r.columns:
        r = r.rename(columns={"start_station_id": "station_id"})
    if "station_name" not in r.columns and "start_station_name" in r.columns:
        r = r.rename(columns={"start_station_name": "station_name"})
    
    r["trips"] = pd.to_numeric(r.get("trips", pd.NA), errors="coerce")
    
    # Find available radii
    avail_radii = []
    for c in r.columns:
        mm = re.match(r"^crashes_within_(\d+)m$", str(c))
        if mm:
            avail_radii.append(int(mm.group(1)))
    
    if not avail_radii:
        print("ERROR: No crashes_within_<R>m columns found")
        r = None
    else:
        # Choose radius
        env_raw = os.environ.get("AXA_RADIUS", "auto")
        wanted = parse_radius_to_m(env_raw)
        radius_used = max(avail_radii) if wanted == -1 else (wanted if wanted in avail_radii else (500 if 500 in avail_radii else max(avail_radii)))
        
        crash_col = f"crashes_within_{radius_used}m"
        
        # Ensure crash column is numeric
        if crash_col not in r.columns:
            r[crash_col] = 0
        r[crash_col] = pd.to_numeric(r[crash_col], errors="coerce").fillna(0).astype(int)
        
        # Compute RAW rate
        r["risk_rate"] = (r[crash_col] / r["trips"].replace({0: np.nan})) * SCALE
        r["risk_rate"] = pd.to_numeric(r["risk_rate"], errors="coerce").fillna(0.0)
        risk_metric_name = f"Raw crash rate per 100k trips ({radius_used}m radius)"

else:
    print("ERROR: No data available (neither scorecard nor risk file)")
    r = None

# Proceed with plotting if we have data
if r is not None:
    # Clean data
    r = r.dropna(subset=["trips", "risk_rate"]).copy()
    r = r[r["trips"] > 0].copy()
    r = r[r["trips"] >= MIN_TRIPS_FOR_ZONES].copy()
    
    if len(r) == 0:
        print(f"No stations meet trips ≥ {MIN_TRIPS_FOR_ZONES} — skipping zones plot.")
    else:
        print(f"Plotting {len(r):,} stations with ≥ {MIN_TRIPS_FOR_ZONES:,} trips")
        
        x = r["trips"].astype(float)
        y = r["risk_rate"].astype(float)
        
        x_med = float(np.nanmedian(x))
        y_med = float(np.nanmedian(y))
        
        print(f"Median exposure: {x_med:,.0f} trips")
        print(f"Median risk: {y_med:.2f}")
        
        # Classify into zones
        def zone(row):
            hi_x = float(row["trips"]) >= x_med
            hi_y = float(row["risk_rate"]) >= y_med
            if hi_x and hi_y:
                return "High exposure / High risk"
            if hi_x and (not hi_y):
                return "High exposure / Lower risk"
            if (not hi_x) and hi_y:
                return "Lower exposure / High risk"
            return "Lower exposure / Lower risk"
        
        r["zone"] = r.apply(zone, axis=1)
        
        # Show zone distribution
        print("\nZone distribution:")
        display(r["zone"].value_counts().to_frame("stations"))
        
        # Plot quadrants
        plt.figure(figsize=(10, 7))
        
        # Define colors for each zone
        zone_colors = {
            "High exposure / High risk": "#d62728",      # red
            "High exposure / Lower risk": "#2ca02c",     # green
            "Lower exposure / High risk": "#ff7f0e",     # orange
            "Lower exposure / Lower risk": "#1f77b4"     # blue
        }
        
        for z, sub in r.groupby("zone"):
            plt.scatter(
                sub["trips"], 
                sub["risk_rate"], 
                alpha=0.6, 
                s=60,
                label=f"{z} (n={len(sub)})",
                color=zone_colors.get(z, "#999999")
            )
        
        # Add median lines
        plt.axvline(x_med, linestyle="--", color="gray", alpha=0.7, linewidth=1.5, label=f"Median exposure ({x_med:,.0f})")
        plt.axhline(y_med, linestyle="--", color="gray", alpha=0.7, linewidth=1.5, label=f"Median risk ({y_med:.2f})")
        
        # Formatting
        plt.xscale("log")
        plt.xlabel("Exposure (trips, log scale)", fontsize=11)
        plt.ylabel(risk_metric_name, fontsize=11)
        
        source_label = "EB-smoothed" if use_scorecard else "Raw data"
        plt.title(
            f"Exposure vs Risk Zones — {source_label}\n"
            f"(stations with ≥ {MIN_TRIPS_FOR_ZONES:,} trips, mode={mode})",
            fontsize=12,
            fontweight="bold"
        )
        
        plt.legend(fontsize=8, loc="best")
        plt.grid(True, alpha=0.3, linestyle=":")
        plt.tight_layout()
        
        # Save
        filename = f"10_zones_exposure_vs_risk_{radius_used}m_trips_ge_{MIN_TRIPS_FOR_ZONES}_{'EB' if use_scorecard else 'raw'}.png"
        savefig(filename)
        plt.show()
        
        # Table of "high-high" quadrant (prevention priority)
        hh = r[(x >= x_med) & (y >= y_med)].copy()
        hh = hh.sort_values(["risk_rate", "trips"], ascending=False).head(15)
        
        print(f"\n HIGH PRIORITY STATIONS (High exposure + High risk, top 15):")
        display_cols = ["mode", "station_id", "station_name", "trips", "risk_rate"]
        if "crash_count" in hh.columns:
            display_cols.append("crash_count")
        if "credibility_flag" in hh.columns:
            display_cols.append("credibility_flag")
        
        display(hh[[c for c in display_cols if c in hh.columns]])
        
        # Additional insights
        if use_scorecard:
            print("\n EB Smoothing Info:")
            if "eb_m_prior_used" in r.columns:
                m_val = r["eb_m_prior_used"].iloc[0]
                print(f"  Prior strength (m): {m_val:,.0f}")
            if "scoring_strategy" in r.columns:
                strat = r["scoring_strategy"].iloc[0]
                print(f"  Strategy: {strat}")
        
        print(f"\n Business Interpretation:")
        print(f"  • {len(hh)} high-priority stations → Prevention pilots & safety interventions")
        high_exp_low_risk = len(r[(x >= x_med) & (y < y_med)])
        print(f"  • {high_exp_low_risk} high-exposure/low-risk stations → Product upsell targets")

else:
    print("  Skipping zones plot due to data issues")

## 7) AXA Partner Decision Assets: Where + When + What

This section turns your outputs into **two decision assets**:

- **WHERE to focus** (stations): `axa_partner_scorecard_500m.csv`
- **WHEN to activate** (time windows): `axa_target_windows.csv`

Both files are produced by the Make targets you already ran.


In [ ]:
# --- AXA Decision Assets (INSURER-READY): WHERE + WHEN + WHAT (clean tables, no booleans/percentile columns) ---
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np

MIN_TRIPS = 5000
TOP_N = 20

# -----------------------
# Helpers: load if missing
# -----------------------
_SCORE_RE = re.compile(r"^axa_partner_scorecard_(\d+)m\.csv$")

def _available_scorecards(run_dir: Path) -> list[int]:
    radii = []
    for p in run_dir.glob("axa_partner_scorecard_*m.csv"):
        m = _SCORE_RE.match(p.name)
        if m:
            radii.append(int(m.group(1)))
    return sorted(set(radii))

def _pick_scorecard_path(run_dir: Path) -> Path | None:
    radii = _available_scorecards(run_dir)
    if not radii:
        return None
    chosen = 500 if 500 in radii else max(radii)
    return run_dir / f"axa_partner_scorecard_{chosen}m.csv"

def _safe_read(path: Path) -> pd.DataFrame | None:
    try:
        return pd.read_csv(path) if path.exists() else None
    except Exception as e:
        print(f"ERROR reading {path}: {e}")
        return None

def _drop_bool_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Remove True/False columns for presentation (keep data intact in df_score)."""
    bool_cols = [c for c in df.columns if df[c].dropna().isin([True, False]).all()]
    # also catch dtype=bool
    bool_cols += [c for c in df.columns if str(df[c].dtype) == "bool"]
    bool_cols = sorted(set(bool_cols))
    return df.drop(columns=bool_cols, errors="ignore")

# -----------------------
# Ensure inputs exist
# -----------------------
if "RUN_DIR" in globals() and RUN_DIR is not None:
    run_dir = Path(RUN_DIR)
else:
    run_dir = Path(".").resolve()

if "df_score" not in globals() or df_score is None:
    score_path = _pick_scorecard_path(run_dir)
    df_score = _safe_read(score_path) if score_path else None

if "df_windows" not in globals() or df_windows is None:
    df_windows = _safe_read(run_dir / "axa_target_windows.csv")

# -----------------------
# Generate decision assets
# -----------------------
if df_score is None or df_windows is None:
    print("Missing inputs:")
    print("  df_score:", "OK" if df_score is not None else "MISSING (scorecard CSV not found)")
    print("  df_windows:", "OK" if df_windows is not None else "MISSING (axa_target_windows.csv not found)")
    print("RUN_DIR used:", run_dir)
else:
    score = df_score.copy()
    win = df_windows.copy()

    # Defensive typing
    for c in ["exposure_trips","crash_count","risk_rate_per_100k_trips",
              "eb_risk_rate_per_100k_trips","expected_incidents_proxy","axa_priority_score"]:
        if c in score.columns:
            score[c] = pd.to_numeric(score[c], errors="coerce")

    # Ensure credibility flag exists
    if "credibility_flag" not in score.columns:
        score["credibility_flag"] = np.where(
            score.get("exposure_trips", 0) >= MIN_TRIPS, "credible", "insufficient_data"
        )

    # station columns
    id_col   = "start_station_id" if "start_station_id" in score.columns else ("station_id" if "station_id" in score.columns else None)
    name_col = "start_station_name" if "start_station_name" in score.columns else ("station_name" if "station_name" in score.columns else None)

    # -----------------------------
    # WHERE (Prevention): credible only, rank by burden (risk × exposure)
    # -----------------------------
    prevention = score[score["credibility_flag"] == "credible"].copy()

    if "expected_incidents_proxy" in prevention.columns:
        prevention = prevention.sort_values(["expected_incidents_proxy","exposure_trips"], ascending=False)
        where_rank_label = "expected burden (risk × exposure)"
    else:
        prevention = prevention.sort_values(["axa_priority_score","exposure_trips"], ascending=False)
        where_rank_label = "axa_priority_score"

    # Presentation columns (NO percentiles, NO booleans)
    where_cols = [c for c in [
        "mode","year","month",id_col,name_col,
        "exposure_trips","crash_count",
        "eb_risk_rate_per_100k_trips",
        "expected_incidents_proxy",
        "axa_priority_score",
        "credibility_flag"
    ] if c and (c in prevention.columns)]

    # Drop any leftover True/False columns just in case
    prevention_show = _drop_bool_cols(prevention[where_cols].copy())

    print(f"\nWHERE (Prevention) — Top {TOP_N} credible station-periods by {where_rank_label}")
    print(f"Credibility rule: exposure_trips ≥ {MIN_TRIPS:,}\n")
    display(prevention_show.head(TOP_N))

    print("Prevention pool size:", f"{len(prevention):,}", "rows (credible)")
    if "expected_incidents_proxy" in prevention.columns:
        print("Total expected burden (sum expected_incidents_proxy):",
              f"{prevention['expected_incidents_proxy'].sum():,.1f}")

    # -----------------------------
    # WHERE (Reach): top exposure (marketing / enrollment reach)
    # -----------------------------
    reach = score.sort_values(["exposure_trips"], ascending=False).copy()

    reach_cols = [c for c in [
        "mode","year","month",id_col,name_col,
        "exposure_trips",
        "credibility_flag"
    ] if c and (c in reach.columns)]

    reach_show = _drop_bool_cols(reach[reach_cols].copy())

    print(f"\nWHERE (Reach) — Top {TOP_N} station-periods by exposure_trips")
    display(reach_show.head(TOP_N))

    # -----------------------------
    # WHEN: activation windows (rank by priority_metric if present)
    # -----------------------------
    for c in ["trips","pct_of_mode_year_trips","pct_within_week_part","priority_metric"]:
        if c in win.columns:
            win[c] = pd.to_numeric(win[c], errors="coerce")

    w_rank = "priority_metric" if "priority_metric" in win.columns else ("trips" if "trips" in win.columns else None)
    when = win.sort_values(w_rank, ascending=False).head(TOP_N) if w_rank else win.head(TOP_N)

    # Keep WHEN table simple
    when_cols = [c for c in [
        "window_type","segment","window_label",
        "trips","pct_of_mode_year_trips","priority_metric",
        "recommended_action"
    ] if c in when.columns]

    when_show = _drop_bool_cols(when[when_cols].copy())

    print(f"\nWHEN — Top {TOP_N} activation windows by {w_rank}")
    display(when_show)

    # -----------------------------
    # WHAT: short insurer pitch
    # -----------------------------
    print("\nWHAT (insurer-ready plan):")
    print(
        f"- Prevention: prioritize credible stations (≥{MIN_TRIPS:,} trips) ranked by expected burden (risk × exposure).\n"
        f"- Timing: activate in the highest-traffic windows (ranked by {w_rank}) to maximize reach and conversion.\n"
        f"- Measurement: short-term conversion lift; mid-term change in incident proxy at treated stations; long-term claims frequency."
    )


In [ ]:
# # --- What m was actually used? ---

# if "eb_m_prior_used" in df_score.columns:
#     print("=== m values by scope ===\n")
    
#     m_info = df_score.groupby(["mode"])["eb_m_prior_used"].agg(["first", "min", "max", "count"])
#     display(m_info)
    
#     for mode in df_score["mode"].unique():
#         m_val = df_score[df_score["mode"] == mode]["eb_m_prior_used"].iloc[0]
#         n_stations = len(df_score[df_score["mode"] == mode])
        
#         print(f"\nMode: {mode}")
#         print(f"  Stations: {n_stations:,}")
#         print(f"  m value: {m_val:,.0f}")

## Where are the generated plots and tables?

Figures are saved under:

- `reports/<RUN_TAG>/figures/`

Key files:
- `09_exposure_vs_risk_zones.png`
- `station_zones.csv`
- `axa_partner_scorecard_500m.csv`
- `axa_target_windows.csv`
